In [ ]:
import pandas as pd
import glob
import numpy as np
import copy
import matplotlib.pyplot as plt

In [ ]:
time_col = 'TIME'
position_cols = ['BPOGX', 'BPOGY']
reading_col = 'FPOGID'
fixation_col = 'Fixation'
fixation_id_col = 'Fixation_ID'
velocity_col = 'Velocity'
distances_col = 'Distances'
periods_col = 'Periods'
# new pos cols
abs_position_cols = ['ABS_X', 'ABS_Y']

In [ ]:
pixel_width = 1920
pixel_height = 1080
params = [pixel_width, pixel_height]

In [ ]:
dfs = {}
for file in glob.glob("data/*/*text*.csv"):
    df = pd.read_csv(file)
    df_cols = df.columns.values
    df_cols[4] = 'TIME'
    df.columns = df_cols
    dfs[file] = df[position_cols + [reading_col] + [time_col]]
    
for filename, df in dfs.items():
    print(filename)
    for a, r, p in zip(abs_position_cols, position_cols, params):
        df[a] = df[r]*p

<h3>Calculate distances between points and velocity

In [ ]:
diff_cols = ['DIFF_'+c for c in abs_position_cols]
for filename, df in dfs.items():
    for d, c in zip(diff_cols, abs_position_cols):
        df[d] = df[c].diff().shift(-1)
    df[periods_col] = df[time_col].diff().shift(-1)
    df[distances_col] = np.square(df[diff_cols]).sum(axis=1).pow(1./2)
    df[velocity_col] = df[distances_col] / df[periods_col]
    df = df.fillna(method='ffill')